In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import json
import sys

In [7]:
path = os.getcwd()

folders = []

for r, d, f in os.walk(path):
    for folder in d:
        if folder == ".ipynb_checkpoints":
            continue
        else:
            print(folder)
            folders.append(os.path.join(r, folder))

for f in folders:
    print(f)

Dial_Left_to_Right
Dial_Right_to_Left
Down_to_up
Left_to_right
Point
Right_to_Left
Up-to-Down
C:\Users\OWNER\Documents\Projects\God\Smart-Home-Project\Machine Learning\Dial_Left_to_Right
C:\Users\OWNER\Documents\Projects\God\Smart-Home-Project\Machine Learning\Dial_Right_to_Left
C:\Users\OWNER\Documents\Projects\God\Smart-Home-Project\Machine Learning\Down_to_up
C:\Users\OWNER\Documents\Projects\God\Smart-Home-Project\Machine Learning\Left_to_right
C:\Users\OWNER\Documents\Projects\God\Smart-Home-Project\Machine Learning\Point
C:\Users\OWNER\Documents\Projects\God\Smart-Home-Project\Machine Learning\Right_to_Left
C:\Users\OWNER\Documents\Projects\God\Smart-Home-Project\Machine Learning\Up-to-Down


In [8]:
ds_move = []
ds_var = [];
ds_std = [];
ds_slp = [];
ds_mean = []

for x in range(len(folders)):
   temp = os.listdir(folders[x])
   os.chdir(folders[x])
   print(x)
   #print("Temp is changing folders to ", temp)
   for c in range(len(os.listdir(folders[x]))):
       #if not c.strip(): continue  # skip the empty line
       #print(str(x) + 'aya');
       dataset = pd.read_csv(temp[c], sep=",", header=None);
       dataset.columns = ["acc_x", "acc_y", "acc_z", "gyr_x", "gyr_y", "gyr_z", "mag_x", "mag_y", "mag_z"];
       dataset = dataset.drop(["mag_x", "mag_y", "mag_z"], axis = 1)
       ds_move.append(x);
       ds_mean.append((dataset.mean()/10000).tolist());
       ds_var.append(dataset.var().tolist());
       ds_std.append(dataset.std().tolist());
       ds_slp.append(dataset.apply(lambda x: np.polyfit(dataset.index, x, 1)[0]).tolist());


0
1
2
3
4
5
6


In [9]:
test_1 = ds_mean[10]
test_2 = ds_mean[2000]
test_3 = ds_mean[1000]
tests = [test_1, test_2, test_3]

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(ds_mean, ds_move, test_size = 0.25)

In [15]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
GAMMA = 0.5
C = 0.5
classifier = SVC(probability=True) #, gamma=GAMMA, C= C
parameters = {'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
              'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
              }
grid_search = GridSearchCV(estimator=classifier, param_grid=parameters, scoring="accuracy", cv=4)
grid_search = grid_search.fit(X_train, y_train)
df = pd.DataFrame(grid_search.cv_results_, columns=["acc_x", "acc_y", "acc_z", "gyr_x", "gyr_y", "gyr_z", "mag_x", "mag_y", "mag_z"])
print(df)
print(grid_search.cv_results_)
print('\n\n')
print(grid_search.best_params_)
print(grid_search.best_score_)

Empty DataFrame
Columns: [acc_x, acc_y, acc_z, gyr_x, gyr_y, gyr_z, mag_x, mag_y, mag_z]
Index: []
{'mean_fit_time': array([0.05759054, 0.27025247, 0.20793283, 0.26226759, 0.05535913,
       0.13314551, 0.15233189, 0.22288501, 0.05583334, 0.07653838,
       0.13462752, 0.2553075 , 0.05534744, 0.05685747, 0.12540287,
       0.33297527, 0.05558765, 0.0498758 , 0.12091804, 0.38989818,
       0.05585945, 0.04585016, 0.11892623, 0.44663137, 0.05559701,
       0.0461275 , 0.1191653 , 0.50596356, 0.05596823, 0.04612803,
       0.11857295, 0.51607251, 0.0555883 , 0.04687035, 0.121279  ,
       0.53751099, 0.04811645, 0.23785442, 0.14535809, 0.19295257,
       0.04836529, 0.09597826, 0.10991544, 0.17252755, 0.04761803,
       0.06057763, 0.0984661 , 0.20019859, 0.04786724, 0.05061477,
       0.09423876, 0.26215702, 0.04785401, 0.04637152, 0.09149826,
       0.3138755 , 0.04810697, 0.04562318, 0.09025121, 0.36972064,
       0.04737896, 0.04685777, 0.09072274, 0.42108464, 0.047867  ,
       0.046